# Vanila GAN

In [2]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

## Model on MNIST dataset

In [ ]:
# normalize data

def mnist_data():
    compose = transforms.Compose()